### Get set up

In [ ]:
# IMPORTS
import os
import pandas as pd
import numpy as np
from utils.load_utils import import_brava_data, get_exp_meta, normalise_shear, NumpyEncoder
from utils.data_utils import calculate_best_m_tau, calculate_lyapunov_exponents
import utils.plot_utils as plot
import matplotlib.pyplot as plt
from datetime import datetime
import json

# JUPYTER
# %matplotlib widget
%reload_ext autoreload
%autoreload 2

# FILENAMES
exp_code = "b1378"
run_code = "05"
working_dir = os.getcwd()
data_dir = working_dir + "/data"
results_dir = working_dir + "/results/" + run_code
if not "params" in globals():
    params={}
params.update({
    "working_dir": working_dir,
    "exp_code": exp_code,
    "run_code": run_code,
    "data_filename": data_dir + '/' + exp_code + '.txt',
    "data_picklename": data_dir + '/' + exp_code + '.pickle',
    "results_dir": results_dir
})
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

### Import data

In [ ]:
# IMPORT PARAMS
params.update({
    "exp_meta": get_exp_meta(params),
    "downsample_factor": 10
})

# IMPORT
data = import_brava_data(params)
print("... data imported")

# HAVE A LOOK
print(data.head())
plt.plot(data["TIME"], data["SHEAR STRESS"])

### Generate windows

In [ ]:
win_length = 250000
win_offset = 125000
num_wins = (len(data) - win_length) // win_offset
wins = []
for i in range(num_wins):
    wins.append((i*win_offset, i*win_offset + win_length))
params.update({ "wins": wins })
print(num_wins)

In [ ]:
# turn ROIs into windows
wins = []
for roi in params["exp_meta"]["ROIs"]:
    masked = data[ data["TIME"].between(roi[0], roi[1]) ]
    wins.append((masked.index[0], masked.index[-1]))
params.update({ "wins": wins })

### Runtime params

In [ ]:

params.update({

    # VIEW SOME WINDOWS
    "save_wins": [0,1,2,5,9,10,11],

    # CALCULATE BEST m, tau BY CAO (1997)
    "tau_to_try": np.array([5,15,30,50,80,120,170,250]),
    "m_to_try": np.arange(1,12,1),
    "E1_threshold": 0.9,
    "E2_threshold": 0.9,
    "eps_over_L0": 0.05,
    "n_neighbors": 20,

    # LYAPUNOV CALCULATIONS
    "LEs_sampling": ['rand',None]
})


# The loop

In [ ]:
FLAG_SAVE = True
params.update({ "run_start_time": datetime.now(), "current_loop": 0 })

results = []

for i, win in enumerate(params["wins"]):
    print("WINDOW ", (i+1), "/", len(params["wins"]))
    params.update({"current_loop": i})
    data_ = data[win[0]:win[1]].copy()
    data_ = normalise_shear(data_)

    m, tau = calculate_best_m_tau(data_, params, fig=(i in params["save_wins"]), save=FLAG_SAVE)
    print("Best m = ", m, ", tau=", tau)

    if ~np.isnan(m):
        LEs, KYD = calculate_lyapunov_exponents(data_, params, m, tau)
        print("Lyapunov Exponents:", LEs)
        print("Kaplan-Yorke dimension:", KYD)
    else:
        LEs = []
        KYD = np.nan

    results.append({ "t": data_.iloc[0].TIME, "m": m, "tau": tau, "LEs": LEs, "KYD": KYD })
    print("\n" + ("_"*20) + "\n")

results = pd.DataFrame(results)

plot.dimensionality_vs_time(data, results, params, save=FLAG_SAVE)
plot.tau_vs_time(data, results, params, save=FLAG_SAVE)

params.update({ "run_end_time": datetime.now() })

if FLAG_SAVE:
    results.to_csv(params["results_dir"] + "/results.csv")
    with open(params["results_dir"] + "/params.json", 'w') as f:
        f.write( json.dumps(params, indent=4, cls=NumpyEncoder) )
    